In [ ]:
import pandas as pd
import numpy as np
import re

#read in excel raw data for all conditions- "FILE PATH" is a placeholder to be edited by the user
fname = "FILE PATH"
auc_df = pd.read_excel(fname)

auc_df

In [ ]:
#Transpose dataframes so that the lines are the rows and the measurements are columns 
auc_trans = auc_df.set_index('Unnamed: 0').transpose()
auc_trans

#Drop unnecessary columns
auc_PA = auc_trans['%Area']
num_peaks = auc_trans['Number of Peaks']


#Find the largest % area peak in each line
peaks = np.arange(1, num_peaks.max()+1, 1).astype(int).tolist()
auc_PA.columns = peaks
auc_PA.fillna(value = 0, inplace = True)
auc_PA['Max % Area'] = auc_PA.max(axis=1)
auc_PA['Max Peak'] = auc_PA.idxmax(axis=1).astype(int)

auc_PA


In [ ]:
#Test if peak largest % area peak and largest area peak are the same
auc_area = auc_trans['Area']
auc_area.columns = peaks
auc_area.fillna(value = 0, inplace = True)
auc_area['Max Area'] = auc_area.max(axis=1)
auc_area['Max Peak'] = auc_area.idxmax(axis=1).astype(int)

test = auc_PA['Max Peak'] == auc_area['Max Peak']
False in test

In [ ]:
#Add max peaks to the dataframes with the rest of the data
auc_peaks = auc_trans.drop(columns=['Baseline', 'Total Area', 'Total Peak Area', 'Number of Peaks']).dropna(axis = 'columns', how = 'all')
#auc_peaks['Max % Area'] = auc_PA['Max % Area']
auc_peaks['Max Peak'] = auc_PA['Max Peak']
auc_peaks

In [ ]:
#Deal with duplicate column names
peak_str = [str(i) for i in peaks]
peak_str
f = ['First X_' + sub for sub in peak_str]
l = ['Last X_' + sub for sub in peak_str]
px = ['Peak X_' + sub for sub in peak_str]
py = ['Peak Y_' + sub for sub in peak_str]
a = ['Area_' + sub for sub in peak_str]
pa = ['%Area_' + sub for sub in peak_str]

f_col = {'First X' : f}
l_col = {'Last X' : l}
px_col = {'Peak X' : px}
py_col = {'Peak Y' : py}
a_col = {'Area' : a}
pa_col = {'%Area' : pa}

auc_peaks.rename(columns=lambda c: f_col[c].pop(0) if c in f_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: l_col[c].pop(0) if c in l_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: px_col[c].pop(0) if c in px_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: py_col[c].pop(0) if c in py_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: a_col[c].pop(0) if c in a_col.keys() else c, inplace=True)
auc_peaks.rename(columns=lambda c: pa_col[c].pop(0) if c in pa_col.keys() else c, inplace=True)

auc_peaks

In [ ]:
#Cycle through lines to get a dataframe with only max peak data, searching for numbers that match max peak
c = -1
new_columns = ['First X', 'Last X', 'Peak X', 'Peak Y', 'Area', '%Area', 'Peak']
max_peak_df = pd.DataFrame(columns = new_columns)
for p in auc_peaks['Max Peak']:
    m = []
    c += 1
    line = auc_peaks.index[c]
    for i in auc_peaks.columns:
        match = re.findall('.*_' + str(p) + '$', i)
        match_to_string = ''.join(map(str, match))
        if not match_to_string:
            pass
        else:
            m.append(match_to_string)
    line_max_peak = auc_peaks.loc[line, m]
    max_peak_df_small = pd.DataFrame(line_max_peak).transpose()
    max_peak_df_small['Peak'] = p
    max_peak_df_small.columns = new_columns
    max_peak_df = max_peak_df.append(max_peak_df_small)
max_peak_df
max_peak_df.to_csv('FILE NAME') #"FILE NAME" is a placeholder to be edited by the user